## Scalability Testing

This notebook is for testing purposes.

In [1]:
import random

from config import Color
from helpers import clean
from helpers.connections.mavlink.customtypes.location import ENUPose, GRAPose
from plan import Plan
from simulator import (
    QGC,
    ConfigGazebo,
    ConfigNovis,
    ConfigQGC,
    Gazebo,
    NoVisualizer,
    Simulator,
)

clean()


## Simulation Configuration

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=90) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[
    Color.RED,
    Color.ORANGE,
    Color.GREEN,
    Color.BLUE
    ]

n_uavs_per_gcs = 60
side_len = 10
altitude = 5
max_delay = 3 #sec

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude,heading=0) 
    for _ in base_homes
    ]

colors = [color for color in gcs_colors for _ in range(n_uavs_per_gcs)]

msn_delays=[random.randint(0, max_delay) for _ in base_homes]


## Assign vehicles to GCS (by color)
gcs_sysids = {f"{color.name}_{color.emoji}":
    list(range(i * n_uavs_per_gcs + 1, (i + 1) * n_uavs_per_gcs + 1))
    for i, color in enumerate(gcs_colors)
    }

## Gazebo Configuration

In [3]:
gaz_config = ConfigGazebo(origin = enu_origin,
                        world_path="simulator/gazebo/worlds/runway3.world")

for path,home,c in zip(base_paths,base_homes,colors):
    gaz_config.add(base_path=path,base_home=home,color=c)
#gaz_config.show()

gaz= Gazebo(gaz_config,gra_origin)

## QGroundControl Configuration

In [4]:
qgc_config = ConfigQGC(origin = gra_origin)

for path,home,color,delay in zip(base_paths,base_homes,colors,msn_delays):
    qgc_config.add(base_path=path,base_home=home,color=color,mission_delay=delay)
qgc_config.show()

qgc=QGC(qgc_config)

# No Visualizer

In [5]:
novis_config = ConfigNovis(origin=gra_origin)
for home in base_homes:
    novis_config.add(base_home=home)
novis=NoVisualizer(novis_config)

## Launch Simulator

In [6]:
simulator = Simulator(
	visualizers=[novis],
	gcs_system_ids=gcs_sysids,
	missions=[veh.mission for veh in qgc_config.vehicles],
	terminals=['gcs'],
	verbose=1,
)

orac = simulator.launch()

23:40:04 - Oracle ⚪ - INFO - 🙈 Running without visualization.
23:40:04 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 3111810)
23:40:04 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 3111811)
23:40:04 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 3111812)
23:40:04 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 3111813)


## Oracle checking

In [7]:
orac.run()

23:40:04 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles and 4 GCSs
23:42:01 - Oracle ⚪ - INFO - Received message 'DONE' from GCS BLUE_🟦
23:42:01 - Oracle ⚪ - INFO - GCS BLUE_🟦 removED. Remaining GCS: 3
23:42:02 - Oracle ⚪ - INFO - Received message 'DONE' from GCS GREEN_🟩
23:42:02 - Oracle ⚪ - INFO - GCS GREEN_🟩 removED. Remaining GCS: 2
23:42:02 - Oracle ⚪ - INFO - Received message 'DONE' from GCS ORANGE_🟧
23:42:02 - Oracle ⚪ - INFO - GCS ORANGE_🟧 removED. Remaining GCS: 1
23:42:02 - Oracle ⚪ - INFO - Received message 'DONE' from GCS RED_🟥
23:42:02 - Oracle ⚪ - INFO - GCS RED_🟥 removED. Remaining GCS: 0
23:42:02 - Oracle ⚪ - INFO - ✅ Main monitoring loop completed - all connections closed
23:42:02 - Oracle ⚪ - INFO - 🎉 Oracle shutdown complete!
